In [38]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sqlalchemy
from keys import MySQL_Pass
from keys import OpenWeather_API_key

def create_sql_connection():
    schema = "sql_escooter_forecast"
    host = "127.0.0.1"
    user = "root"
    password = MySQL_Pass
    port = 3306
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

    return connection_string

def read_city_table_from_sql(connection_str):
    cities_df = pd.read_sql('SELECT city_id, city FROM cities', connection_str)
    return cities_df

def get_weather_info(cities_df):
    API_key = OpenWeather_API_key
    city_name = []
    forecast_time = []
    rain_in_last_3h = []
    outlook = []
    temp = []
    feels_like = []
    rain_prob = []
    wind_speed = []
    city_id = []
    
    for index, row in cities_df.iterrows():
        city = row['city']
        weather_data = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={'Berlin'}&appid={'5da7f5090cb9b90f1bfa7a1674c0f4a2'}&units=metric")
        weather_data_json = weather_data.json()
        
        #print(weather_data_json)  # Print the JSON response
        
        weather_list = weather_data_json.get("list", [])  # Use .get() to handle missing keys
        
        for i, dt in enumerate(weather_list):
            forecast_time.append(weather_list[i]["dt"])
            temp.append(weather_list[i]['main']["temp"])
            feels_like.append(weather_list[i]['main']["feels_like"])
            rain_prob.append(weather_list[i]['pop'])
            #rain_in_last_3h.append(weather_list[i]['rain']['3h'])
            wind_speed.append(weather_list[i]['wind']['speed'])
            outlook.append(weather_list[i]['weather'][0]["description"])
            city_name.append(city)
            city_id.append(cities_df['city_id'][index])
            
    weather_df = pd.DataFrame(list(zip(city_id, forecast_time, outlook,temp,  feels_like, wind_speed , rain_prob)), 
                                  columns=["city_id", "forecast_time", "outlook", "temperature",  "feels_like", "wind_speed", "rain_prob"])
    weather_df['forecast_time'] = pd.to_datetime(weather_df['forecast_time'], unit='s')
    outlook_text = weather_df.iloc[0]['outlook']    
    return weather_df

def push_weather_data_to_sql(weather_df, conn_str):
    outlook_text = weather_df.iloc[11]['outlook']
    weather_df.to_sql('weather',
                      if_exists='append',
                      con=conn_str,
                      index=False)
    return f'Done Successfully! {outlook_text}'

In [42]:
con_str = create_sql_connection()
city_df = read_city_table_from_sql(con_str)
weather_df = get_weather_info(city_df)
push_weather_data_to_sql(weather_df, con_str)

'Done Successfully! light rain'

In [25]:
city_df

,city_id,city
0,1,Berlin


In [26]:
city_df['city']

0    Berlin
Name: city, dtype: object